# 흐름 : Access ID 호출 -> 매치 ID 조회 및 호출 -> 경기 상세정보 조회

### NEXON API 호출 (Live version api)

In [1]:
import requests
import pandas as pd

# 라이브 API 키
API_KEY = "live_7a611a04eeb1ac043f43a92245935f274608d65acac4fcb584f1baad81aa8bd7efe8d04e6d233bd35cf2fabdeb93fb0d"

# 헤더 수정 (중요!)
headers = {
    'x-nxopen-api-key': API_KEY
}

# 예시: 유저 accessId 조회
nickname = "급할수록도라ae몽"  # 여기에 조회할 유저 닉네임 넣어
url = "https://open.api.nexon.com/fconline/v1/id"

params = {
    "nickname": nickname
}

response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    user_data = response.json()
    print(f"✅ 유저 accessId 조회 성공: {user_data}")
else:
    print(f"❌ 상태 코드: {response.status_code}")
    print(f"❌ 유저 accessId 조회 실패: {response.text}")


✅ 유저 accessId 조회 성공: {'ouid': '40260d503f67f41c85ad1fbb6bf97fae'}


### matchtype dataframe

In [2]:
import pandas as pd

# FC 온라인 matchtype 코드 + 의미 정의
matchtype_data = [
    {"matchtype": 40, "desc": "친선경기 (클래식 1:1 매치 포함)"},
    {"matchtype": 50, "desc": "공식경기"},
    {"matchtype": 52, "desc": "감독모드 공식경기"},
    {"matchtype": 60, "desc": "VOLTA LIVE"},
    {"matchtype": 61, "desc": "VOLTA AI"},
    {"matchtype": 80, "desc": "VOLTA 커스텀 매치"},
    {"matchtype": 81, "desc": "VOLTA 커리어"},
]

# DataFrame 변환
df_matchtype = pd.DataFrame(matchtype_data)

# 출력
print(df_matchtype)

   matchtype                  desc
0         40  친선경기 (클래식 1:1 매치 포함)
1         50                  공식경기
2         52             감독모드 공식경기
3         60            VOLTA LIVE
4         61              VOLTA AI
5         80          VOLTA 커스텀 매치
6         81             VOLTA 커리어


### 최근 매치 조회 (10회)

In [2]:
import requests

# 1️⃣ API KEY 및 헤더 설정
API_KEY = "live_7a611a04eeb1ac043f43a92245935f274608d65acac4fcb584f1baad81aa8bd7efe8d04e6d233bd35cf2fabdeb93fb0d"
headers = {
    'x-nxopen-api-key': API_KEY
}

# 2️⃣ 유저 ouid (앞에서 조회한 것)
ouid = "40260d503f67f41c85ad1fbb6bf97fae"

# 3️⃣ API URL (v1/user/match 사용!)
url_matches = "https://open.api.nexon.com/fconline/v1/user/match"

# 4️⃣ 쿼리 파라미터
params = {
    "ouid": ouid,
    "matchtype": 40,   # 친선경기 (클래식 1:1 포함)
    "offset": 0,
    "limit": 10
}

# 5️⃣ 요청 전송
response_matches = requests.get(url_matches, headers=headers, params=params)

# 6️⃣ 결과 확인
if response_matches.status_code == 200:
    match_ids = response_matches.json()
    print(f"✅ 최근 match ID 리스트 (총 {len(match_ids)}건):")
    for i, match_id in enumerate(match_ids, 1):
        print(f"{i}. {match_id}")
elif response_matches.status_code == 204:
    print("⚠️ 경기 기록 없음 (204 No Content)")
else:
    print(f"❌ 상태 코드: {response_matches.status_code}")
    print(f"❌ 경기 리스트 조회 실패: {response_matches.text}")


✅ 최근 match ID 리스트 (총 10건):
1. 684d989285790f517bbd9434
2. 684d95c0f8e3247a910db042
3. 684d92e6313e8da3e4f86d37
4. 6849808e9c9ea33922d75174
5. 68497e2632dbd6b99a778382
6. 68497b9fe3944aeb03edca06
7. 684978fb36ac4dc10ab65c34
8. 68493c7202abea6ac506a786
9. 684939f809588314ae9d1a53
10. 6847a771f02c4dd03a1bb7c9


In [ ]:
import requests
import json

API_KEY = "live_7a611a04eeb1ac043f43a92245935f274608d65acac4fcb584f1baad81aa8bd7efe8d04e6d233bd35cf2fabdeb93fb0d"
headers = {
    'x-nxopen-api-key': API_KEY
}

# 예시: 최근 match_id 하나 넣어보기
match_id = "684d95c0f8e3247a910db042"   # 여기 원하는 match_id 넣으면 됨

# API URL
url_match_detail = f"https://open.api.nexon.com/fconline/v1/match-detail?matchid={match_id}"

# 요청 전송
response_match_detail = requests.get(url_match_detail, headers=headers)

# 결과 확인
if response_match_detail.status_code == 200:
    match_detail = response_match_detail.json()
    # 예쁘게 출력 (json pretty print)
    print("✅ 경기 상세 정보:")
    print(json.dumps(match_detail, indent=2, ensure_ascii=False))
elif response_match_detail.status_code == 204:
    print("⚠️ 해당 match_id 에 대한 상세 기록 없음 (204 No Content)")
else:
    print(f"❌ 상태 코드: {response_match_detail.status_code}")
    print(f"❌ 경기 상세 조회 실패: {response_match_detail.text}")


✅ 경기 상세 정보:
{
  "matchId": "6849808e9c9ea33922d75174",
  "matchDate": "2025-06-11T13:21:43",
  "matchType": 40,
  "matchInfo": [
    {
      "ouid": "40260d503f67f41c85ad1fbb6bf97fae",
      "nickname": "급할수록도라ae몽",
      "matchDetail": {
        "seasonId": null,
        "matchResult": "무",
        "matchEndType": 0,
        "systemPause": 0,
        "foul": 0,
        "injury": 0,
        "redCards": 0,
        "yellowCards": 0,
        "dribble": 52,
        "cornerKick": 3,
        "possession": 46,
        "offsideCount": 1,
        "averageRating": 3.96667,
        "controller": "keyboard"
      },
      "shoot": {
        "shootTotal": 3,
        "effectiveShootTotal": 3,
        "shootOutScore": 0,
        "goalTotal": 1,
        "goalTotalDisplay": 1,
        "ownGoal": 0,
        "shootHeading": 0,
        "goalHeading": 0,
        "shootFreekick": 0,
        "goalFreekick": 0,
        "shootInPenalty": 3,
        "goalInPenalty": 1,
        "shootOutPenalty": 0,
        "goa

In [7]:
import requests
import json
import pandas as pd

# 1️⃣ API KEY 및 헤더 설정
API_KEY = "live_7a611a04eeb1ac043f43a92245935f274608d65acac4fcb584f1baad81aa8bd7efe8d04e6d233bd35cf2fabdeb93fb0d"
headers = {
    'x-nxopen-api-key': API_KEY
}

# 2️⃣ 조회할 match_id
match_id = "684d95c0f8e3247a910db042"

# 3️⃣ API URL (query param 방식 사용)
url_match_detail = f"https://open.api.nexon.com/fconline/v1/match-detail?matchid={match_id}"

# 4️⃣ 요청 전송
response_match_detail = requests.get(url_match_detail, headers=headers)

# 5️⃣ 결과 확인 및 파싱
if response_match_detail.status_code == 200:
    match_detail = response_match_detail.json()
    
    # matchInfo 리스트 순회
    parsed_data = []
    for info in match_detail["matchInfo"]:
        ouid = info.get("ouid")
        nickname = info.get("nickname")
        detail = info["matchDetail"]
        
        match_result = detail.get("matchResult")
        possession = detail.get("possession")
        corner_kick = detail.get("cornerKick")
        offside_count = detail.get("offsideCount")
        average_rating = detail.get("averageRating")
        controller = detail.get("controller")
        
        # 경기 요약 dict 구성
        parsed_data.append({
            "matchId": match_detail.get("matchId"),
            "matchDate": match_detail.get("matchDate"),
            "matchType": match_detail.get("matchType"),
            "ouid": ouid,
            "nickname": nickname,
            "matchResult": match_result,
            "possession (%)": possession,
            "cornerKick": corner_kick,
            "offsideCount": offside_count,
            "averageRating": average_rating,
            "controller": controller
        })
    
    # DataFrame 생성
    df_match_summary = pd.DataFrame(parsed_data)
    
    # 출력
    from IPython.display import display

    display(df_match_summary)

    
    # 필요시 CSV로 저장 가능
    # df_match_summary.to_csv("match_summary.csv", index=False, encoding="utf-8-sig")

elif response_match_detail.status_code == 204:
    print("⚠️ 해당 match_id 에 대한 상세 기록 없음 (204 No Content)")
else:
    print(f"❌ 상태 코드: {response_match_detail.status_code}")
    print(f"❌ 경기 상세 조회 실패: {response_match_detail.text}")


,matchId,matchDate,matchType,ouid,nickname,matchResult,possession (%),cornerKick,offsideCount,averageRating,controller
0,684d95c0f8e3247a910db042,2025-06-14T15:41:23,40,6fcf2b3f3ac52bf388e3cc9a1bba1f68,2500하세영,승,45,2,1,4.28889,keyboard
1,684d95c0f8e3247a910db042,2025-06-14T15:41:23,40,40260d503f67f41c85ad1fbb6bf97fae,급할수록도라ae몽,패,55,4,0,4.00000,keyboard
